I am creating the transductive dataset using only one part of the entire dataset

In [1]:
import pandas as pd
import numpy as np
import tarfile

seed = 1234

In [2]:
 # open file
file = tarfile.open(f'MSCallGraph_0.tar.gz')

    # extracting file
file.extractall('./')

file.close()
df = pd.read_csv(f'MSCallGraph_0.csv')
df=df.drop(['Unnamed: 0','timestamp','rpcid','interface','rt','traceid'],axis=1)
df = df.replace('(?)', np.NaN)
df = df.replace('', np.NaN)
df = df.replace('NAN', np.NaN)
df = df.dropna()
df = df.drop_duplicates()
df.nunique()

um         1474
rpctype       6
dm         7326
dtype: int64

In [3]:
df

,um,rpctype,dm
0,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,mc,b1dbd3a649a3cc790fa12573c9c1aa00988e07a8818a22...
11,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,fd6d86bd0fd550e717c1fdb82a33190a9fef216d87d535...
20,5cca70246befb1f4c9546d2912b9419dee54439218efa5...,userDefined,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
26,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,84f9f68ef003a21288fffe8f9a09a5a29b05f4cc4229b8...
42,4ab265f54516248ee8873be7d6441912456ce17e84f399...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
...,...,...,...
6081607,0674d4c8b48f44a836019adff9dffe0cce2ee6e7b93ee0...,db,039ff41dc3636105ffd26b0ec675eec246a965e8102c77...
6081615,1cf86b19c487cc10ea3bb9623a319454029e040d0f4e9d...,db,bb61ac88608568ac2017c8559c1da973f4f94c7d1a958e...
6081625,2c52c868f36f3920d201fddda80c8062998ebc07ee2b85...,db,abb42d7303a4376befc681f9b4a765c6b6575de0df98ed...
6081642,1cf86b19c487cc10ea3bb9623a319454029e040d0f4e9d...,db,b0748973546ca26a9cbfe7802e52924291a76929a704a4...


In [4]:
entities = set(df['dm']) | set(df['um'])
len(entities)

7386

In [5]:
baseline = []
for entity in entities:
    baseline.append(df[df['dm'] == entity].head(1))
    baseline.append(df[df['um'] == entity].head(1))

In [6]:
baseline_df = pd.concat(baseline)
baseline_df = baseline_df.drop_duplicates()
baseline_df

,um,rpctype,dm
35274,0696938ae3212896c34626b3816d92ca5b0bed0cfda981...,db,6c489e6dd8ca17efc85582e577773c31bf56e425b3f5ea...
2140906,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,d0d3a580e84abf4295331c292dabd0eb2350c307b26da9...
2140905,d0d3a580e84abf4295331c292dabd0eb2350c307b26da9...,db,f1256f9958dfe9d4e377f492e30960098bc1dae1f1bc96...
1179360,829140e0701cea23ef10f12bac3ccc3704e8fcb46a2a79...,rpc,6471bd9933c7d7f263d6317bb70394e46c644c86031cfc...
1179347,6471bd9933c7d7f263d6317bb70394e46c644c86031cfc...,rpc,829140e0701cea23ef10f12bac3ccc3704e8fcb46a2a79...
...,...,...,...
2318409,7b3b9cb2faed94c69ae18e8275ec186a4ffe6b2e258878...,mc,eb921ac016d05c727a47628600b5ddfe83c2fb1f81c562...
283485,d03bb97862607468fe3153b28d41a20de1e3144a566264...,db,3e5965c47685bb9a6d59a937aa047aaf3c721320307f72...
5113408,c20853fda9525f7fa31660361104c878c275e77f9ff7bf...,db,d99a50f1e4bef33ec02611e5b3e536d80f8b0286a8aee2...
1612620,fb750d9e38225fe305f254bb00cfba22138845328abf15...,db,a0f6b90ca61e2d99ed4a0f0978b6b850c58820bb9071a5...


In [7]:
n_train_items = int(len(df)*0.64)
n_items_to_add = n_train_items - len(baseline_df)
print("the number of items to add to the train set is: ",n_items_to_add)

the number of items to add to the train set is:  2185


In [8]:
not_baseline_df = df.merge(baseline_df, how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only'].drop('_merge',axis= 1)
not_baseline_df

,um,rpctype,dm
9,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,userDefined,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...
11,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
12,4ab265f54516248ee8873be7d6441912456ce17e84f399...,http,091794afdcf1abbaf62404337ff72a38df96c5d8af8bd0...
19,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,e6c95bef37d936ebb375bf135ded88c96eb9257a1d3d9d...
20,4ab265f54516248ee8873be7d6441912456ce17e84f399...,rpc,5cca70246befb1f4c9546d2912b9419dee54439218efa5...
...,...,...,...
16208,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...,db,a2a5a9a0492f495f3dcc0bd933b0819cc8ac14289059d0...
16209,5019702f16bab0ec5627583db3cf7def36cbd87674088b...,db,6115254f36ed41a883d751b18e2bf78cc1c1ec76cd3da2...
16211,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,1abd717ffef1e3721f131eb3b37fc5c84fb9c52d0a42f3...
16213,9a0f0dd3e278f678cfa3f874a7c74af8206de653ec0396...,rpc,5ef671a7799d52d75c5c1b87d3542b5619c0ca7c9b3d80...


In [9]:
not_baseline_train = not_baseline_df.sample(n_items_to_add,random_state = seed)
train_df = pd.concat([baseline_df, not_baseline_train])
train_df

,um,rpctype,dm
35274,0696938ae3212896c34626b3816d92ca5b0bed0cfda981...,db,6c489e6dd8ca17efc85582e577773c31bf56e425b3f5ea...
2140906,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,d0d3a580e84abf4295331c292dabd0eb2350c307b26da9...
2140905,d0d3a580e84abf4295331c292dabd0eb2350c307b26da9...,db,f1256f9958dfe9d4e377f492e30960098bc1dae1f1bc96...
1179360,829140e0701cea23ef10f12bac3ccc3704e8fcb46a2a79...,rpc,6471bd9933c7d7f263d6317bb70394e46c644c86031cfc...
1179347,6471bd9933c7d7f263d6317bb70394e46c644c86031cfc...,rpc,829140e0701cea23ef10f12bac3ccc3704e8fcb46a2a79...
...,...,...,...
5560,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,rpc,4ed7c6414695b486064dc4611c135492c4459ce9f3edbd...
12393,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...,db,c4295080f8c580f6c929788ab68d53622b899d1d3de90d...
3164,aa42d395e6b2ab2146c4f60800b1463b0b4eec59147ff5...,mc,8f8da7913c052992423534e5b9f49e72b4213ea8fd10db...
10578,377f1c6e2b6c3315132d0d8dbd721dc01630e5919f39b0...,rpc,f10612cc80a4777bcb0f9025041b3e4a9d8de8b20bfd2d...


In [10]:
remaining_df = not_baseline_df.drop(not_baseline_train.index) 

In [11]:
validation_df = remaining_df.sample(frac = 0.44, random_state = seed)
validation_df

,um,rpctype,dm
6921,99f2e7b501f50db9b4089242a5d3e1aba334c32e0c718b...,mq,2a8dcc1d92fecd5d42d5be4a5d6c443a97b377526ad778...
12980,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,635a68af6f78ae2c3ca0832f23f3f5dfbfb54200b00e7e...
16191,87f3522e7b259f45398e495f02e0afd54d9bd8f77959c6...,db,ce22bc7ea1cf2f5c71c3202976d7a4e3f10d74494efa40...
209,4d980b8ae1f1da1a302d77ef91b9f9dd4e841856f6cdd4...,mc,0b0c981da8042a436f30f7d9f7cce74360abe630bd5478...
10200,2888a351e30955adb09e56e906c415036ab8b10f9ebc16...,http,2bbca6dc6ba99115e631eba25efc62743aff6d9fe3fe3a...
...,...,...,...
13051,e2350413f1606f6332ee2a2d98f09029cd454b0b216b24...,rpc,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...
6395,48e8faba4a168f803d7f16abb37ff877edd589b8081166...,db,6e02c6f25dc4d71560b03ba9077de4cb37383a1ec809de...
1596,3557add998c8762136da6e3a0af6489ed6425c3d91ca9e...,rpc,648b2a0c2f0a2c60f552c18abbc9ac747657751fc8db49...
5701,24601dd8b36f856eb0a4d759866f475bc62cf843ffe092...,rpc,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...


In [12]:
test_df = remaining_df.drop(validation_df.index)  # drop the sampled rows to get the second DataFram
test_df

,um,rpctype,dm
51,95a6f7f8345e2eca31ee74ddc19d547e7fc0f5c8e65772...,rpc,3cab0a98767379fbcd059750c07ec42dbd0374cb114e0f...
57,e5155edb237fd9a04d9b322383113d21db13ade01944a5...,mc,676ca482c56de84046ce1d0076abd6cdb0f25778d036db...
67,95a6f7f8345e2eca31ee74ddc19d547e7fc0f5c8e65772...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
70,75e56c8fbb9336eb4dd40f5f609d5344203d374d73fd0b...,mc,01d660afcfadafd587e20ec4c04ddbc7eb0de95643ba0e...
77,63b8a25a4a9f80c7508c0db0d163af6d9e6c0746c9b7e2...,mc,6746853e39c5008988baf677bf27c846ee99173cafdec2...
...,...,...,...
16205,b39889af09ab89f4a7918a534348e405b52260bf198a43...,rpc,80df1ad95e41fc8fdef7671b6dd89e6b7ee5c982038a07...
16208,f2ff782d57371c090c294859e444ee4bdfb4b317e68368...,db,a2a5a9a0492f495f3dcc0bd933b0819cc8ac14289059d0...
16211,5b563df180af0e674902ffbeddf022580eee216f0a172f...,db,1abd717ffef1e3721f131eb3b37fc5c84fb9c52d0a42f3...
16213,9a0f0dd3e278f678cfa3f874a7c74af8206de653ec0396...,rpc,5ef671a7799d52d75c5c1b87d3542b5619c0ca7c9b3d80...


Transductive checks

In [13]:
train_entities = set(train_df['dm']) or set(train_df['um'])
valid_entities = set(validation_df['dm']) or set(validation_df['um'])
test_entities = set(test_df['dm']) or set(test_df['um'])

In [14]:
assert(valid_entities.issubset(train_entities))

In [15]:
assert(test_entities.issubset(train_entities))

In [16]:
train_relations = set(train_df['rpctype'])
valid_relations = set(validation_df['rpctype'])
test_relations = set(test_df['rpctype'])

In [17]:
assert(valid_relations.issubset(train_relations))

In [18]:
assert(test_relations.issubset(train_relations))

Check to see if there is test leakage

In [19]:
assert(0==len(pd.merge(train_df, validation_df)))

In [20]:
assert(0==len(pd.merge(train_df, test_df)))

In [21]:
assert(0==len(pd.merge(test_df, validation_df)))

In [22]:
def create_triplets(df):
    triplets = []
    for i in range(len(df)):
        head = df.iloc[i]['dm']
        tail = df.iloc[i]['um']
        rel = df.iloc[i]['rpctype']
        triplets.append([head,rel,tail])
    return triplets

In [23]:
import asposecells
import jpype
jpype.startJVM()

from asposecells.api import Workbook, FileFormatType

def create_tsv(triplets, file_name):


    # Create Workbook object.
    workbook = Workbook(FileFormatType.TSV)

    # Access the first worksheet of the workbook.
    worksheet = workbook.getWorksheets().get(0)

    # Get the desired cell(s) of the worksheet and input the value into the cell(s).


    i = 1
    for elem in triplets: 
        worksheet.getCells().get("A"+str(i)).putValue(elem[0])
        worksheet.getCells().get("B"+str(i)).putValue(elem[1])
        worksheet.getCells().get("C"+str(i)).putValue(elem[2])
        i+=1


    # Save the workbook as TSV file.
    workbook.save(file_name)


    file = open(file_name,'r')  
    lines = file.readlines()  
    file.close()
    file = open(file_name,'w')  
    lines = lines[:-1]
    file.writelines(lines)
    file.close()

In [24]:
datasets = [train_df, validation_df, test_df]
names = ["train", "validation","test"]

In [25]:
for elem in zip(datasets,names):
    create_tsv(create_triplets(elem[0]),f"MSCallGraph_0_{elem[1]}_transductive.tsv")
jpype.shutdownJVM()